# Исследование статистики чемпионата мира по автогонкам класса Формулы-1 с 1950 по 2017 год

In [133]:
from pandas import Series, DataFrame, read_csv
import pandas as pd
import numpy as np
import datetime

## Для начала общая статистика

In [238]:
seasons = read_csv('dataset/f1_seasons.csv')
races = read_csv('dataset/f1_races.csv')
drivers = read_csv('dataset/f1_drivers.csv')
races = races.loc[(races['date'] < datetime.datetime.today().strftime('%Y-%m-%d'))]
print 'Общее количество проведенных чемпионатов: {}'.format(seasons.shape[0])
print 'Общее количество проведенных гонок: {}'.format(races.shape[0])
print 'Общее количество гонщиков выходило на старт: {}'.format(drivers.shape[0])

Общее количество проведенных чемпионатов: 68
Общее количество проведенных гонок: 970
Общее количество гонщиков выходило на старт: 840


## А теперь изучим географию

### Сколько всего стран принимали этапы чемпионата мира

In [141]:
circuits = read_csv('dataset/f1_circuits.csv')
races = races.merge(circuits, left_on='circuit', right_on='name', how='left')
races = races[['year', 'country', 'location']]
print 'Итого: {}'.format(races.country.unique().shape[0])

Итого: 32


### Сколько всего городов принимали этапы чемпионата мира

In [142]:
print 'Итого: {}'.format(races.location.unique().shape[0])

Итого: 69


### Сделаем группировку по количеству Гранд-при в каждой стране

In [202]:
races.country.value_counts()

Italy           95
Germany         76
UK              71
USA             67
Monaco          64
Belgium         62
France          59
Spain           54
Canada          48
Brazil          44
Japan           34
Australia       33
Hungary         32
Netherlands     30
Austria         30
South Africa    23
Argentina       20
Malaysia        18
Mexico          17
Portugal        16
China           14
Bahrain         13
Singapore       10
UAE              8
Turkey           7
Sweden           6
Switzerland      5
Korea            4
Russia           4
India            3
Azerbaijan       2
Morocco          1
Name: country, dtype: int64

Как видно, больше всего Ф-1 любит приезжать в Италию, а в Марокко, видимо, заехали по случайности.

### Сделаем группировку по наиболее популярным городам Гранд-при

In [216]:
cities = races.groupby(['location', 'country'])['location'] \
    .agg(['count']) \
    .sort_values('count', ascending=False)
cities.head(10)

,,count
location,country,
Monza,Italy,67
Monte-Carlo,Monaco,64
Silverstone,UK,51
Spa,Belgium,50
Nürburg,Germany,40
Montreal,Canada,38
Hockenheim,Germany,35
São Paulo,Brazil,34
Budapest,Hungary,32


Как видно, ни один город не принимал этапы все 68 сезонов подряд, но итальянский город Монца был к этому очень близок.

### Для наглядности - 10-ка наименее популярных городов

In [217]:
cities.tail(10)

,,count
location,country,
Baku,Azerbaijan,2
Castle Donington,UK,1
Dallas,USA,1
Le Mans,France,1
Styria,Austria,1
Casablanca,Morocco,1
Lisbon,Portugal,1
Florida,USA,1
Pescara,Italy,1


Если у Баку, как действующему этапу, удастся выйти из этого списка, то в остальных городах появление Ф-1 в ближайшее время не предвидится.

## С географией более-менее разобрались, можно переходить к гонщикам

### Общее количество стран, которые представляли гонщики

In [239]:
print 'Итого: {}'.format(drivers.nationality.unique().shape[0])

Итого: 41


### Посмотрим статистику распределения гонщиков по странам

In [240]:
drivers.nationality.value_counts()

British              162
American             157
Italian               99
French                72
German                49
Brazilian             31
Argentine             24
Belgian               23
South African         23
Swiss                 23
Japanese              19
Australian            17
Dutch                 17
Spanish               15
Austrian              15
Canadian              13
Swedish               10
Finnish                9
New Zealander          8
Mexican                6
Danish                 5
Irish                  5
Uruguayan              4
Rhodesian              4
Portuguese             4
Colombian              3
East German            3
Venezuelan             3
Monegasque             3
Russian                2
Indian                 2
Czech                  1
Argentine-Italian      1
American-Italian       1
Chilean                1
Polish                 1
Liechtensteiner        1
Hungarian              1
Indonesian             1
Malaysian              1


С большим отрывом лидируют британцы и американцы, но такое количество американцев можно объяснить наличием гонки Инди 500, которая формально входила в чемпионат в 50-е годы, проверим это предположение:

In [241]:
results = read_csv('dataset/f1_results.csv')

drivers_results = results.merge(drivers, on='driverRef', how='inner')

drivers_results_50 = drivers_results[['nationality', 'driverRef']] \
    .loc[(drivers_results['year'] >= 1950) & (drivers_results['year'] <= 1960)]

drivers_results_50 = drivers_results_50.drop_duplicates()

drivers_results_50.nationality.value_counts()

American             128
British               82
Italian               29
French                26
German                21
Argentine             16
Belgian               13
Swiss                  8
Spanish                4
Brazilian              4
Uruguayan              4
Australian             4
East German            3
Dutch                  3
Monegasque             2
Argentine-Italian      1
Portuguese             1
Venezuelan             1
New Zealander          1
Irish                  1
American-Italian       1
Thai                   1
Swedish                1
Name: nationality, dtype: int64

Как видим, наша догадка оказалась точна, большинство американцев выступало в Ф-1 с 1950 по 1960 годы, врочем, как и почти половина британцев.

### Перейдем уже к гонщикам и посмотрим кто и сколько побеждал в автогонках

In [242]:
drivers_results[drivers_results.position == 1].driver.value_counts()

Michael Schumacher      91
Lewis Hamilton          60
Alain Prost             51
Sebastian Vettel        46
Ayrton Senna            41
Fernando Alonso         32
Nigel Mansell           31
Jackie Stewart          27
Niki Lauda              25
Jim Clark               25
Juan Fangio             24
Nico Rosberg            23
Nelson Piquet           23
Damon Hill              22
Mika Häkkinen           20
Kimi Räikkönen          20
Stirling Moss           16
Jenson Button           15
Emerson Fittipaldi      14
Graham Hill             14
Jack Brabham            14
Alberto Ascari          13
David Coulthard         13
Mario Andretti          12
Alan Jones              12
Carlos Reutemann        12
Felipe Massa            11
Jacques Villeneuve      11
Rubens Barrichello      11
James Hunt              10
                        ..
Pat Flaherty             1
Lee Wallard              1
Jean-Pierre Beltoise     1
Johnnie Parsons          1
Vittorio Brambilla       1
Rodger Ward              1
S

Как видно, больше всего побед у Михаэля Шумахера (но это и так все знают). Посмотрим, кто рекордсмен из действующих гонщиков:

In [243]:
drivers_2017 = drivers_results[drivers_results.year==2017].driverRef.drop_duplicates()
drivers_results[(drivers_results.position == 1) & (drivers_results.driverRef.isin(drivers_2017))] \
    .driver.value_counts()

Lewis Hamilton      60
Sebastian Vettel    46
Fernando Alonso     32
Kimi Räikkönen      20
Jenson Button       15
Felipe Massa        11
Daniel Ricciardo     5
Valtteri Bottas      2
Max Verstappen       1
Name: driver, dtype: int64

Лидирует Льюис Хэмильтон, который в общем списке на втором месте.

### Построим небольшую статистику по победителям

In [290]:
total_winners = drivers_results[drivers_results.position==1].driverRef.drop_duplicates().shape[0]
print 'Итого: {}'.format(total_winners)
print 'Соотношение участников/победителей: {}'.format(
    float(drivers_results.driverRef.drop_duplicates().shape[0]) / float(total_winners))

Итого: 107
Соотношение участников/победителей: 7.85046728972


Получается, что каждый 8 гонщик имеет шансы хотя бы на одну победу в ГП.

### Окей, с победителями понятно, теперь проведем исследование по чемпионским титулам